# Importação das Bibliotecas

In [1]:
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import math
import json
import ipywidgets as widgets
import pyperclip
import os
import pystac_client
import planetary_computer
import rich.table
import odc.stac
import tqdm
from tqdm.auto import tqdm as progss
import requests
from landsatxplore.api import API
from landsatxplore.earthexplorer import EarthExplorer

# Definições de parametros de entrada

In [2]:
_posMaps = [-9.7083, -39.8532]
_lado = 5 #Tamanho do lado do quadrado em km
_mes_de_referencia = 1
_ano_inicial = 2020
_ano_final = 2023
_NomeLocal = "Poço de Fora, Curaçá - BA"
_filtroNuvem = 10 #Excluir tudo que tiver mais que 10% de nuvens
output_dir = 'landsat_data'
bands_of_interest = ["nir08", "red", "green", "blue", "lwir11", "lwir"]
_login = "gladistony.silva"
_senha = "96884544Gladis@"
#lwir11 pode está nomeado como em alguns dados lwir 

# Funções de conversão e processamento de informações

In [3]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
dir_saida = output_dir+"/"+f"{_posMaps[0]} {_posMaps[1]} {_lado}/"
if not os.path.exists(dir_saida):
    os.makedirs(dir_saida)
def kmToGrau(km, valor):
    return valor + (km / (111.32 * math.cos(math.radians(valor))))
def calcular_vertices(lat, lon):
    metade = _lado / 2
    lat1 = kmToGrau(-metade, lat)
    lon1 = kmToGrau(-metade, lon)
    lat2 = kmToGrau(-metade, lat)
    lon2 = kmToGrau(metade, lon)
    lat3 = kmToGrau(metade, lat)
    lon3 = kmToGrau(metade, lon)
    lat4 = kmToGrau(metade, lat)
    lon4 = kmToGrau(-metade, lon)
    return [[lat1, lon1], [lat2, lon2], [lat3, lon3], [lat4, lon4]]
mes_string = "{:02d}".format(_mes_de_referencia)
def dias_mes(mes, ano):
  data = datetime.date(ano, mes, 1)
  proximo_mes = data.replace(month=data.month % 12 + 1)
  ultimo_dia = proximo_mes - datetime.timedelta(days=1)
  return ultimo_dia.day
_intervalo_de_tempo = []
for i in range(_ano_inicial,_ano_final+1,1):
    dia_final = dias_mes(_mes_de_referencia, i)
    _intervalo_de_tempo.append(f"{i}-{mes_string}-01/{i}-{mes_string}-{dia_final}")
def downloadinfo(file_url, name):
    response = requests.get(file_url, stream=True)
    file_size = int(response.headers.get("content-length", 0))
    progress = tqdm.tqdm(total=file_size, unit="B", unit_scale=True)
    with open(name, "wb") as f:
        for chunk in response.iter_content(chunk_size=1024):
            f.write(chunk)
            progress.update(len(chunk))
    progress.close()

# Exibição primaria dos resultados
As informações de coordenadas podem ser melhor visualizadas no seguinte site:

https://geojson.io/

Nele foi feito a seguinte demarcação do mapa para ser utilizado:

In [4]:
link = "https://geojson.io/#map=8/"+str(round(_posMaps[0],3))+"/"+str(round(_posMaps[1],3))
print(link)

https://geojson.io/#map=8/-9.708/-39.853


Delimitar uma area no mapa que será usada para fazer a busca por imagens, para isso criar um quadrado usando essas posições como centro

In [5]:
quadrado = calcular_vertices(_posMaps[1],_posMaps[0])
print(quadrado)

[[-39.88245375253269, -9.731084068863375], [-39.88245375253269, -9.685515931136624], [-39.82394624746731, -9.685515931136624], [-39.82394624746731, -9.731084068863375]]


In [6]:
data = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            quadrado[0],
            quadrado[1],
            quadrado[2],
            quadrado[3],
            quadrado[0]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}
json_string = json.dumps(data, indent=2)
print(json_string)
button = widgets.Button(description="Copiar GeJson")
def on_button_clicked(b):
    pyperclip.copy(json_string)
button.on_click(on_button_clicked)
button

{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              -39.88245375253269,
              -9.731084068863375
            ],
            [
              -39.88245375253269,
              -9.685515931136624
            ],
            [
              -39.82394624746731,
              -9.685515931136624
            ],
            [
              -39.82394624746731,
              -9.731084068863375
            ],
            [
              -39.88245375253269,
              -9.731084068863375
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}


Button(description='Copiar GeJson', style=ButtonStyle())

Ajustando as variaveis, para deixar conforme o padrão informado no site: https://planetarycomputer.microsoft.com/docs/quickstarts/reading-stac/

In [7]:
_area_de_interesse = {
    "type": "Polygon",
    "coordinates": [
          [
            quadrado[0],
            quadrado[1],
            quadrado[2],
            quadrado[3],
            quadrado[0]
          ]
        ]
}
print(_area_de_interesse)
bbox_of_interest = _area_de_interesse['coordinates']
bbox_of_interest = [round(bbox_of_interest[0][0][0],2),round(bbox_of_interest[0][0][1],2),round(bbox_of_interest[0][2][0],2),round(bbox_of_interest[0][2][1],2)]
print( bbox_of_interest)

{'type': 'Polygon', 'coordinates': [[[-39.88245375253269, -9.731084068863375], [-39.88245375253269, -9.685515931136624], [-39.82394624746731, -9.685515931136624], [-39.82394624746731, -9.731084068863375], [-39.88245375253269, -9.731084068863375]]]}
[-39.88, -9.73, -39.82, -9.69]


Tentar fazer login na api e baixar as informações

In [8]:
api = API(_login, _senha)
scenes = api.search(
    dataset='landsat_tm_c2_l1',
    bbox = bbox_of_interest,
    start_date='1995-01-01',
    end_date='1995-10-01',
    max_cloud_cover=100
)
print(f"{len(scenes)} scenes found.")
api.logout()

10 scenes found.


In [9]:
print(scenes[0])

{'cloud_cover': 32, 'entity_id': 'LT52170671995229CUB00', 'display_id': 'LT05_L1TP_217067_19950817_20200912_02_T1', 'ordering_id': 'None', 'landsat_product_id': 'LT05_L1TP_217067_19950817_20200912_02_T1', 'landsat_scene_id': 'LT52170671995229CUB00', 'acquisition_date': datetime.datetime(1995, 8, 17, 0, 0), 'collection_category': datetime.datetime(2023, 12, 1, 0, 0), 'collection_number': 2, 'sensor_mode': 'SAM', 'wrs_path': 217, 'wrs_row': 67, 'full_partial_scene': 'FULL', 'date_product_generated': datetime.datetime(2020, 9, 12, 0, 0), 'start_time': datetime.datetime(1995, 8, 17, 11, 51, 30, 703250), 'stop_time': datetime.datetime(1995, 8, 17, 11, 51, 57, 681880), 'station_id': 'CUB', 'day-night_indicator': 'DAY', 'ground_control_points_model': 262, 'land_cloud_cover': 32.0, 'scene_cloud_cover': 32.0, 'ground_control_points_version': 5, 'geometric_rmse_model': 4.733, 'geometric_rmse_model_x': 3.204, 'geometric_rmse_model_y': 3.484, 'sensor_anomalies': 'NONE', 'data_anomaly': None, 'imag

In [10]:
ee = EarthExplorer(_login, _senha)

ee.download('LT51960471995178MPS00', output_dir='./data', show_progress=True)

ee.logout()

TypeError: download() got an unexpected keyword argument 'show_progress'